In [1]:
%cd /content/drive/MyDrive/
# raw_data is imported from global config

/content/drive/MyDrive


In [2]:
%%capture
! pip install mne==0.19.2;
! pip install torch==1.7.0;
! pip install hdf5storage;
! pip install skorch==0.10.0;
! pip install Braindecode==0.5.1;

***Copy and Paste your code below.***

In [3]:
import hdf5storage
import h5py
import os
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import torch
from torch import nn
from gesture.models.resnet import _my_resnet18, my_resnet18
from skorch.callbacks import LRScheduler
from skorch.helper import predefined_split
from braindecode import EEGClassifier

from gesture.config import *
from common_dl import myDataset, set_random_seeds
from myskorch import on_epoch_begin_callback, on_batch_end_callback


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

Attempting to create new mne-python configuration file:
/root/.mne/mne-python.json


In [4]:
from gesture.models.resnet import my_resnet18

In [5]:
sid = 10  # 4

data_dir = data_dir + 'preprocessing/P' + str(sid) + '/tfInput/'
filename=data_dir+'dataset.hdf5'
f1 = h5py.File(filename, "r")
list(f1.keys())
X_train = f1['X_train'][:] # (650, 10, 148, 250)
X_test = f1['X_test'][:] #(50, 10, 148, 250)
y_train = f1['y_train'][:] # (650,)
y_test = f1['y_test'][:]
f1.close()

In [6]:
train_ds = myDataset(X_train, y_train)
val_ds = myDataset(X_test, y_test)
train_loader = DataLoader(dataset=train_ds, batch_size=1, shuffle=True, pin_memory=False)
val_loader = DataLoader(dataset=val_ds, batch_size=1, pin_memory=False)
#test it
#(x,y)=iter(train_loader).next() # x: torch.Size([1, 10, 148, 250]); y: torch.Size([1])


In [7]:
cuda = torch.cuda.is_available()
device = 'cuda' if cuda else 'cpu'
if cuda:
    torch.backends.cudnn.benchmark = True
seed = 20200220
set_random_seeds(seed=seed) # same as braindecode random seeding

In [8]:
_net=_my_resnet18(10,5, pretrained=True).float()
net=my_resnet18(_net)
if cuda:
    net.cuda()
#x= torch.randn(1, 10, 148, 250)
#net(x).shape

lr = 0.0001
weight_decay = 1e-10
batch_size = 1
n_epochs = 100

In [9]:
location=os.getcwd()
if re.compile('/Users/long/').match(location):
    my_callbacks=[
        "accuracy", ("lr_scheduler", LRScheduler('CosineAnnealingLR', T_max=n_epochs - 1)),
        ('on_epoch_begin_callback', on_epoch_begin_callback),('on_batch_end_callback',on_batch_end_callback),
    ]
elif re.compile('/content/drive').match(location):
   my_callbacks=[
        "accuracy", ("lr_scheduler", LRScheduler('CosineAnnealingLR', T_max=n_epochs - 1)),
    ]


In [10]:
clf = EEGClassifier(
    net,
    criterion=torch.nn.NLLLoss,  #torch.nn.NLLLoss/CrossEntropyLoss
    optimizer=torch.optim.Adam, #optimizer=torch.optim.AdamW,
    train_split=predefined_split(val_ds),  # using valid_set for validation; None means no validate:both train and test on training dataset.
    optimizer__lr=lr,
    optimizer__weight_decay=weight_decay,
    batch_size=batch_size,
    callbacks=my_callbacks,
    device=device,
)

In [11]:
clf.fit(train_ds, y=None, epochs=n_epochs)

/content/drive/MyDrive/gesture/models/resnet.py:57: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.last_layer(x)


  epoch    train_accuracy    train_loss    valid_accuracy    valid_loss      lr      dur
-------  ----------------  ------------  ----------------  ------------  ------  -------
      1            0.2000        1.6734            0.2000        1.6195  0.0001  12.5800
      2            0.2000        1.6572            0.2000        1.6148  0.0001  12.3214
      3            0.2000        1.6460            0.2000        1.6588  0.0001  12.4837
      4            0.2000        1.6405            0.2000        1.6262  0.0001  12.2705
      5            0.2000        1.6441            0.2000        1.6369  0.0001  12.5427
      6            0.2000        1.6377            0.2000        1.6430  0.0001  12.3061


<class 'braindecode.classifier.EEGClassifier'>[initialized](
  module_=my_resnet18(
    (pretrained): ResNet(
      (conv1): Conv2d(10, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1